In [23]:
import pandas as pd
from pyomo.environ import *

# Cargar datos (asegúrate de que df tiene las columnas correctas)
df = pd.read_csv(r'datos_talleres.csv') 

In [24]:
df

,Garage Number,Garage Name,Staff,Show room space,Population in category 1,Population in category 2,Enquiries Alpha model,Enquiries Beta model,Alpha sales,Beta sales,Profit
0,1,Winchester,7,8.0,10,12,8.5,4.0,2.00,0.60,1.50
1,2,Andover,6,6.0,20,30,9.0,4.5,2.30,0.70,1.60
2,3,Basingstoke,2,3.0,40,40,2.0,1.5,0.80,0.25,0.50
3,4,Poole,14,9.0,20,25,10.0,6.0,2.60,0.86,1.90
4,5,Woking,10,9.0,10,10,11.0,5.0,2.40,1.00,2.00
5,6,Newbury,24,15.0,15,13,25.0,1.9,8.00,2.60,4.50
6,7,Portsmouth,6,7.0,50,40,8.5,3.0,2.50,0.90,1.60
7,8,Alresford,8,7.5,5,8,9.0,4.0,2.10,0.85,2.00
8,9,Salisbury,5,5.0,10,10,5.0,2.5,2.00,0.65,0.90
9,10,Guildford,8,10.0,30,35,9.5,4.5,2.05,0.75,1.70


In [25]:
 # Definir insumos y productos
inputs = ['Staff', 'Show room space', 'Population in category 1', 'Population in category 2', 'Enquiries Alpha model', 'Enquiries Beta model']
outputs = ['Alpha sales', 'Beta sales', 'Profit']
DMUs = df['Garage Name'].tolist()

# Crear modelo para cada DMU
results = {}
for k in DMUs:
    model = ConcreteModel()
    
    # Variables de peso
    model.u = Var(outputs, domain=NonNegativeReals)
    model.v = Var(inputs, domain=NonNegativeReals)
    
    # Función objetivo (Maximizar la eficiencia del taller k)
    model.obj = Objective(
        expr=sum(df.loc[df['Garage Name'] == k, o].values[0] * model.u[o] for o in outputs),
        sense=maximize
    )
    
    # Restricciones de eficiencia para todos los talleres
    model.constraints = ConstraintList()
    for j in DMUs:
        model.constraints.add(
            sum(df.loc[df['Garage Name'] == j, o].values[0] * model.u[o] for o in outputs) 
            <= sum(df.loc[df['Garage Name'] == j, i].values[0] * model.v[i] for i in inputs)
        )
    
    # Normalización (denominador de la función de eficiencia = 1)
    model.norm = Constraint(
        expr=sum(df.loc[df['Garage Name'] == k, i].values[0] * model.v[i] for i in inputs) == 1
    )
    
    # Resolver
    solver = SolverFactory('glpk')  # Asegúrate de tener GLPK instalado
    solver.solve(model)
    
    # Guardar resultados
    results[k] = {
        'Efficiency': sum(df.loc[df['Garage Name'] == k, o].values[0] * model.u[o].value for o in outputs),
        'u': {o: model.u[o].value for o in outputs},
        'v': {i: model.v[i].value for i in inputs}
    }

# Convertir resultados en DataFrame
results_df = pd.DataFrame.from_dict(results, orient='index')
print(results_df)


             Efficiency                                                  u  \
Winchester     0.835396  {'Alpha sales': 0.113198996497834, 'Beta sales...   
Andover        0.917379  {'Alpha sales': 0.398860398860399, 'Beta sales...   
Basingstoke    1.000000  {'Alpha sales': 1.25, 'Beta sales': 0.0, 'Prof...   
Poole          0.864467  {'Alpha sales': 0.0651289289214115, 'Beta sale...   
Woking         0.845000  {'Alpha sales': 0.0, 'Beta sales': 0.217669299...   
Newbury        1.000000  {'Alpha sales': 0.125, 'Beta sales': 0.0, 'Pro...   
Portsmouth     1.000000  {'Alpha sales': 0.0, 'Beta sales': 0.953744684...   
Alresford      1.000000  {'Alpha sales': 0.0, 'Beta sales': 0.267950356...   
Salisbury      1.000000  {'Alpha sales': 0.5, 'Beta sales': 0.0, 'Profi...   
Guildford      0.802439  {'Alpha sales': 0.0619833804437647, 'Beta sale...   
Alton          1.000000  {'Alpha sales': 0.0, 'Beta sales': 1.428571428...   
Weybridge      0.854345  {'Alpha sales': 0.379436964504284, 'Bet

In [26]:
eficientes = results_df[results_df['Efficiency'] == 1]
ineficientes = results_df[results_df['Efficiency'] < 1]
print("Talleres eficientes:")
print(eficientes)
print("Talleres ineficientes:")
print(ineficientes)

Talleres eficientes:
            Efficiency                                                  u  \
Newbury            1.0  {'Alpha sales': 0.125, 'Beta sales': 0.0, 'Pro...   
Salisbury          1.0  {'Alpha sales': 0.5, 'Beta sales': 0.0, 'Profi...   
Henley             1.0  {'Alpha sales': 0.0, 'Beta sales': 0.0, 'Profi...   
Maidenhead         1.0  {'Alpha sales': 0.0, 'Beta sales': 0.5, 'Profi...   

                                                            v  
Newbury     {'Staff': 0.0, 'Show room space': 0.0, 'Popula...  
Salisbury   {'Staff': 0.159914017717561, 'Show room space'...  
Henley      {'Staff': 0.0191739422431161, 'Show room space...  
Maidenhead  {'Staff': 0.0495867768595041, 'Show room space...  
Talleres ineficientes:
             Efficiency                                                  u  \
Winchester     0.835396  {'Alpha sales': 0.113198996497834, 'Beta sales...   
Andover        0.917379  {'Alpha sales': 0.398860398860399, 'Beta sales...   
Basingstoke    1

In [27]:
eficientes

,Efficiency,u,v
Newbury,1.0,"{'Alpha sales': 0.125, 'Beta sales': 0.0, 'Pro...","{'Staff': 0.0, 'Show room space': 0.0, 'Popula..."
Salisbury,1.0,"{'Alpha sales': 0.5, 'Beta sales': 0.0, 'Profi...","{'Staff': 0.159914017717561, 'Show room space'..."
Henley,1.0,"{'Alpha sales': 0.0, 'Beta sales': 0.0, 'Profi...","{'Staff': 0.0191739422431161, 'Show room space..."
Maidenhead,1.0,"{'Alpha sales': 0.0, 'Beta sales': 0.5, 'Profi...","{'Staff': 0.0495867768595041, 'Show room space..."


In [28]:
ineficientes

,Efficiency,u,v
Winchester,0.835396,"{'Alpha sales': 0.113198996497834, 'Beta sales...","{'Staff': 0.0124265446060867, 'Show room space..."
Andover,0.917379,"{'Alpha sales': 0.398860398860399, 'Beta sales...","{'Staff': 0.108618233618234, 'Show room space'..."
Basingstoke,1.000000,"{'Alpha sales': 1.25, 'Beta sales': 0.0, 'Prof...","{'Staff': 0.428571428571429, 'Show room space'..."
Poole,0.864467,"{'Alpha sales': 0.0651289289214115, 'Beta sale...","{'Staff': 0.0, 'Show room space': 0.0188451284..."
Woking,0.845000,"{'Alpha sales': 0.0, 'Beta sales': 0.217669299...","{'Staff': 0.0150829089095783, 'Show room space..."
Guildford,0.802439,"{'Alpha sales': 0.0619833804437647, 'Beta sale...","{'Staff': 0.0143328989864454, 'Show room space..."
Weybridge,0.854345,"{'Alpha sales': 0.379436964504284, 'Beta sales...","{'Staff': 0.149326805385557, 'Show room space'..."
Dorchester,0.866522,"{'Alpha sales': 0.109609516098897, 'Beta sales...","{'Staff': 0.0139814411160241, 'Show room space..."
Bridport,0.981572,"{'Alpha sales': 0.059602588140142, 'Beta sales...","{'Staff': 0.0, 'Show room space': 0.0022333737..."
Weymouth,1.000000,"{'Alpha sales': 0.0, 'Beta sales': 1.006447893...","{'Staff': 0.142847942525356, 'Show room space'..."


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Garage Number             28 non-null     int64  
 1   Garage Name               28 non-null     object 
 2   Staff                     28 non-null     int64  
 3   Show room space           28 non-null     float64
 4   Population in category 1  28 non-null     int64  
 5   Population in category 2  28 non-null     int64  
 6   Enquiries Alpha model     28 non-null     float64
 7   Enquiries Beta model      28 non-null     float64
 8   Alpha sales               28 non-null     float64
 9   Beta sales                28 non-null     float64
 10  Profit                    28 non-null     float64
dtypes: float64(6), int64(4), object(1)
memory usage: 2.5+ KB


In [31]:
final_results

,Garage Number,Efficiency,Input_Weight_Staff,Input_Weight_Show room space,Input_Weight_Population in category 1,Input_Weight_Population in category 2,Input_Weight_Enquiries Alpha model,Input_Weight_Enquiries Beta model,Output_Weight_Alpha sales,Output_Weight_Beta sales,Output_Weight_Profit
Petersfield,18.0,1.000000,0.017381,0.000001,0.010546,0.000001,0.029023,0.017044,0.166666,0.000001,0.000001
Romsey,27.0,1.000000,0.012160,0.000001,0.000001,0.000001,0.298081,0.230471,0.000001,0.000001,1.818181
Bournemouth,23.0,1.000000,0.031005,0.005809,0.000001,0.000001,0.000001,0.027128,0.000001,0.234273,0.060833
Maidenhead,25.0,1.000000,0.049584,0.000001,0.000001,0.000001,0.000001,0.057850,0.000001,0.499997,0.000001
Newbury,6.0,1.000000,0.000001,0.000001,0.000001,0.000001,0.000001,0.526267,0.124999,0.000001,0.000001
Salisbury,9.0,1.000000,0.159912,0.000001,0.001315,0.000001,0.000001,0.074907,0.499999,0.000001,0.000001
Henley,24.0,1.000000,0.019176,0.050017,0.000001,0.000001,0.037395,0.055066,0.000001,0.000001,0.499999
Alton,11.0,1.000000,0.060695,0.000001,0.000001,0.000001,0.077256,0.171673,0.000001,1.428568,0.000001
Alresford,8.0,1.000000,0.018566,0.000001,0.000001,0.000962,0.076010,0.039918,0.000001,0.267944,0.386123
Portland,16.0,1.000000,0.052989,0.000001,0.000001,0.000001,0.256165,0.219118,0.000001,1.650501,0.844647
